In [1]:
#Online shopping Customer Segmentation
#**Problem statement**

# 1: Segment (Group/Cluster) customer on basis of RFM (Recency, Frequency, Monetary) 
# 2: Segment (Group/Cluster) Identify trends for Day, Month, Season, Time by Invoice count
# 3.Top 10 selling product 
# 4 Best product at country level.
# 5 FINDING CUSTOMER VALUATION.

In [3]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore') 

In [14]:
df=pd.read_excel("Online_shopping.xlsx")
df.head()

,InvoiceNo,StockCode,lower,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,white hanging heart t-light holder,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,white metal lantern,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,cream cupid hearts coat hanger,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,knitted union flag hot water bottle,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,red woolly hottie white heart.,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [15]:
df.shape

(541909, 9)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   lower        1816 non-null    object        
 3   Description  540455 non-null  object        
 4   Quantity     541909 non-null  int64         
 5   InvoiceDate  541909 non-null  datetime64[ns]
 6   UnitPrice    541909 non-null  float64       
 7   CustomerID   406829 non-null  float64       
 8   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 37.2+ MB


In [7]:
# 1- Total 8 feature and this one is clustering problem so no target variable column is there.Data set consists of 541909 rows
# 2- InvoiceDate column is already in datetime format
# 3- Has null values in few features as well
# 4- In cistomerID coloum we have null values.

In [17]:
df.isnull().sum()/len(df1)*100

InvoiceNo       0.000000
StockCode       0.000000
lower          99.664888
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64

In [18]:
df.duplicated().sum() # Ignoring duplicates

5268

In [19]:
df1=df.copy()

In [20]:
df1=df1.dropna(subset=['CustomerID'])

In [21]:
df1.isnull().sum()

InvoiceNo           0
StockCode           0
lower          405387
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID          0
Country             0
dtype: int64

# 1: Segment (Group/Cluster) customer on basis of RFM (Recency, Frequency, Monetary) 

In [22]:
# Droping columns are insignificant for customer segmentation
df2 = df1.drop(['StockCode','Description', 'Country','lower'], axis = 1)
df2.head()

,InvoiceNo,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,6,2010-12-01 08:26:00,2.55,17850.0
1,536365,6,2010-12-01 08:26:00,3.39,17850.0
2,536365,8,2010-12-01 08:26:00,2.75,17850.0
3,536365,6,2010-12-01 08:26:00,3.39,17850.0
4,536365,6,2010-12-01 08:26:00,3.39,17850.0


In [12]:
df2.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00000,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08000,4.13,38970.0
CustomerID,541909.0,15287.690570,1484.746010,12346.00,14367.00,15287.69057,16255.00,18287.0


In [13]:
#From descriptive statistics we can see that some of the order in quantity column are negative quantity.
#It occurs when customer returns the defective product.
# For total Sale we have to multiply 'Quantity' with 'UnitPrice'
# In two ways we can handle negative quantity entries
#1- Replace the negative quanity by 0 also so that total sale amount will be 0 for that entry
#2- Leave the negative quantity as it is so that total sale will -ve and it will cancel out the +ve sale of that item to balance out

# But at least customer came and buy something so we need to count that sale amount so lets fill -ve qty by 0

In [23]:
df2=df2[(df2["Quantity"]>0)]

In [24]:
df2.shape
# By removing negative and 0 quantity items we have reduced 10624 rows but considered the sales done.

(397924, 5)

In [25]:
#Creating another column by calculating Total Sale (Sale= Qty x UnitPrice)
df2["Total"]=df2["Quantity"]*df2["UnitPrice"]
df2.head()

,InvoiceNo,Quantity,InvoiceDate,UnitPrice,CustomerID,Total
0,536365,6,2010-12-01 08:26:00,2.55,17850.0,15.30
1,536365,6,2010-12-01 08:26:00,3.39,17850.0,20.34
2,536365,8,2010-12-01 08:26:00,2.75,17850.0,22.00
3,536365,6,2010-12-01 08:26:00,3.39,17850.0,20.34
4,536365,6,2010-12-01 08:26:00,3.39,17850.0,20.34


# 5 FINDING CUSTOMER VALUATION - CLTV


For customer evaluation we can think the problem statement as a regression problem.

Customer Life time value(CLTV) is a quantitative analysis and 
one of the most important metric to modern customer centric business senario.

Customer Lifetime Value is the amount of monetary value a customer
will bring to your company during their lifetime, from the first 
to the last purchase. In other words, it is a number that represents 
the total amount spent by a single customer on your products 
or services over their lifespan.

Customer Lifetime Value (CLTV) is the average revenue you can generate from customers over the entire lifetime of their account. 

Based on that specific number, you will have a better sense of how much 
you can spend on customer acquisition, how much you should invest into 
your existing customers, and you can define strategies and a budget 
to increase your Customer Lifetime Value in eCommerce or retain your 
high-value customers.

In [119]:
dfcltv=df2[['InvoiceNo','Quantity','InvoiceDate','UnitPrice','CustomerID','Total']]
dfcltv.head()

,InvoiceNo,Quantity,InvoiceDate,UnitPrice,CustomerID,Total
0,536365,6,2010-12-01 08:26:00,2.55,17850.0,15.30
1,536365,6,2010-12-01 08:26:00,3.39,17850.0,20.34
2,536365,8,2010-12-01 08:26:00,2.75,17850.0,22.00
3,536365,6,2010-12-01 08:26:00,3.39,17850.0,20.34
4,536365,6,2010-12-01 08:26:00,3.39,17850.0,20.34


# CLTV= Average Total Order value * Purchases Frequency / Churn Rate


For CLTV we need to calculate 3 things of each customer

1.Average Total Order value = Monetary/Frequency i.e. Total shopping done/No. of orders placed

2.Average Purchases per year or Purchase Frequency

3.Churn rate

In [120]:
#For finding Average Total Order value we can use RFMScore table
RFMScore.head()

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_sum,RFM_loyality,RFM_ID,Segment,Cluster
0,12346.0,325,1,77183.60,1,1,5,7,Diamond,115,cant_lose,1
1,12347.0,2,7,4310.00,4,2,3,9,Diamond,423,Drive_Loyality,1
2,12348.0,75,4,1797.24,2,1,2,5,Gold,212,new customer,3
3,12349.0,18,1,1757.55,4,1,2,7,Diamond,412,students,1
4,12350.0,310,1,334.40,1,1,1,3,Silver,111,new customer,0


In [121]:
# AVERAGE ORDER VALUE

RFMScore["avg_order_value"]=RFMScore["Monetary"]/RFMScore["Frequency"]
RFMScore.head()

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_sum,RFM_loyality,RFM_ID,Segment,Cluster,avg_order_value
0,12346.0,325,1,77183.60,1,1,5,7,Diamond,115,cant_lose,1,77183.600000
1,12347.0,2,7,4310.00,4,2,3,9,Diamond,423,Drive_Loyality,1,615.714286
2,12348.0,75,4,1797.24,2,1,2,5,Gold,212,new customer,3,449.310000
3,12349.0,18,1,1757.55,4,1,2,7,Diamond,412,students,1,1757.550000
4,12350.0,310,1,334.40,1,1,1,3,Silver,111,new customer,0,334.400000


In ecommerce, companies are battling for attention, customers, and loyalty.
They typically adopt two broad strategic approaches-
1. Customer Acquisition
2. Customer Retention
However, customer acquisition is costly — around five times more than customer retention on average. 

# Repeat Purchase Rate


The repeat purchase rate is the first place to start when you decide to focus more time and energy on existing customers.
The repeat purchase rate (or repeat customer rate), is a metric that calculates the ratio of repeat customers to the overall customer base
This metric is essential because it tells you how poorly or successfully your customer retention strategies are performing.
The retention rate does not consider new customers so it is different from customerretention rate.

Repeat Purchase Rate = The no of customers completed 2 or more purchases during a given time frame / The total number of customers who bought from your company in the same period 

In [122]:
RFMScore[RFMScore["Frequency"]>1].shape[0]

2848

In [123]:
RFMScore.shape[0]

4340

In [124]:
#Repeat Rate
repeat_rate=RFMScore[RFMScore["Frequency"]>1].shape[0]/RFMScore.shape[0] #shape[0] gives the no of customers
repeat_rate

0.656221198156682

# Purchase Frequency

Customer purchase frequency = Total Number of orders ÷ Total Number of unique customers

If you know how frequently customers are coming back to make purchases, 
you’ll have a good understanding of your success at converting customers into repeat clients, 
helping you to make decisions about your customer retention strategy. 

In [125]:
sum(RFMScore["Frequency"])

20770

In [126]:
RFMScore.shape[0]

4340

In [127]:
#Purchase Frquency
purchase_frequency=sum(RFMScore["Frequency"])/RFMScore.shape[0]
purchase_frequency

4.785714285714286

# Churn Rate

eCommerce churn is the term used to describe the number of customers who stop buying from your online store.
The churn rate is the percentage of customers who stop doing business with you over a predetermined period. 
The important thing is that your churn rate remains as low as possible and doesn’t exceed your growth rate.

In [128]:
#ChurnRate
churn_rate=1-repeat_rate
churn_rate

0.343778801843318

In [129]:
purchase_frequency,repeat_rate,churn_rate

(4.785714285714286, 0.656221198156682, 0.343778801843318)

In [132]:
#customer value
RFMScore["CLTV"]=RFMScore["avg_order_value"]*purchase_frequency/churn_rate
RFMScore.head()

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_sum,RFM_loyality,RFM_ID,Segment,Cluster,avg_order_value,CLTV
0,12346.0,325,1,77183.60,1,1,5,7,Diamond,115,cant_lose,1,77183.600000,1.074466e+06
1,12347.0,2,7,4310.00,4,2,3,9,Diamond,423,Drive_Loyality,1,615.714286,8.571304e+03
2,12348.0,75,4,1797.24,2,1,2,5,Gold,212,new customer,3,449.310000,6.254805e+03
3,12349.0,18,1,1757.55,4,1,2,7,Diamond,412,students,1,1757.550000,2.446670e+04
4,12350.0,310,1,334.40,1,1,1,3,Silver,111,new customer,0,334.400000,4.655153e+03


# Returns on customer lifetime = CLTV

profit margin & CLTV
 Lets assume that business making 25% profit

In [136]:
#profit Margin
RFMScore["profit_margin"]=RFMScore["Monetary"]*0.25
RFMScore.head()


,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_sum,RFM_loyality,RFM_ID,Segment,Cluster,avg_order_value,CLTV,profit_margin
0,12346.0,325,1,77183.60,1,1,5,7,Diamond,115,cant_lose,1,77183.600000,1.074466e+06,19295.9000
1,12347.0,2,7,4310.00,4,2,3,9,Diamond,423,Drive_Loyality,1,615.714286,8.571304e+03,1077.5000
2,12348.0,75,4,1797.24,2,1,2,5,Gold,212,new customer,3,449.310000,6.254805e+03,449.3100
3,12349.0,18,1,1757.55,4,1,2,7,Diamond,412,students,1,1757.550000,2.446670e+04,439.3875
4,12350.0,310,1,334.40,1,1,1,3,Silver,111,new customer,0,334.400000,4.655153e+03,83.6000


In [137]:
# Returns on CUSTOMER LIFETIME VALUE
RFMScore["cust_lifetime_value"]=RFMScore["CLTV"]*RFMScore["profit_margin"]

In [141]:
RFMScore2=RFMScore[['CustomerID','Recency','Frequency','Monetary','avg_order_value','CLTV','profit_margin','cust_lifetime_value']]
RFMScore2.head()

,CustomerID,Recency,Frequency,Monetary,avg_order_value,CLTV,profit_margin,cust_lifetime_value
0,12346.0,325,1,77183.60,77183.600000,1.074466e+06,19295.9000,2.073279e+10
1,12347.0,2,7,4310.00,615.714286,8.571304e+03,1077.5000,9.235580e+06
2,12348.0,75,4,1797.24,449.310000,6.254805e+03,449.3100,2.810346e+06
3,12349.0,18,1,1757.55,1757.550000,2.446670e+04,439.3875,1.075036e+07
4,12350.0,310,1,334.40,334.400000,4.655153e+03,83.6000,3.891708e+05


# Building a Regression model

In [143]:
# creation of month year
dfcltv["month_yr"]=dfcltv["InvoiceDate"].apply(lambda x: x.strftime("%b-%Y"))# extract the month and year

dfcltv.head()

,InvoiceNo,Quantity,InvoiceDate,UnitPrice,CustomerID,Total,month_yr
0,536365,6,2010-12-01 08:26:00,2.55,17850.0,15.30,Dec-2010
1,536365,6,2010-12-01 08:26:00,3.39,17850.0,20.34,Dec-2010
2,536365,8,2010-12-01 08:26:00,2.75,17850.0,22.00,Dec-2010
3,536365,6,2010-12-01 08:26:00,3.39,17850.0,20.34,Dec-2010
4,536365,6,2010-12-01 08:26:00,3.39,17850.0,20.34,Dec-2010


In [146]:
sale=dfcltv.pivot_table(index=["CustomerID"],columns=["month_yr"],values="Total",aggfunc="sum",fill_value=0).reset_index()
sale["CLV"]=sale.iloc[:,2:].sum(axis=1)# sum all the months sale


In [147]:
sale.head()

month_yr,CustomerID,Apr-2011,Aug-2011,Dec-2010,Dec-2011,Feb-2011,Jan-2011,Jul-2011,Jun-2011,Mar-2011,May-2011,Nov-2011,Oct-2011,Sep-2011,CLV
0,12346.0,0.00,0.00,0.00,0.00,0.0,77183.60,0.0,0.00,0.0,0.0,0.00,0.00,0.0,77183.60
1,12347.0,636.25,584.91,711.79,224.82,0.0,475.39,0.0,382.52,0.0,0.0,0.00,1294.32,0.0,3673.75
2,12348.0,367.00,0.00,892.80,0.00,0.0,227.44,0.0,0.00,0.0,0.0,0.00,0.00,310.0,1430.24
3,12349.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,1757.55,0.00,0.0,1757.55
4,12350.0,0.00,0.00,0.00,0.00,334.4,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,334.40


Now lets build a regression model for existing customers.
We need a set of dependent (Y) and independent variables (X). 
we shall take last six-month data as independent variables and 
total revenue over three years as a dependent variable.


In [148]:
X=sale[["Sep-2011","Aug-2011","Jul-2011","Dec-2011","Nov-2011","Oct-2011"]]
Y=sale["CLV"]

In [149]:
#SPLIT TRAINING AND TESTING
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2, random_state = 42)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,Y_train)

Y_pred=lr.predict(X_test)


print(lr.intercept_)
print(lr.coef_)

-49.32124274323223
[0.9837198  2.43487669 2.44446529 1.02121234 2.08966068 1.62583706]


In [150]:
from sklearn import metrics

print("R-Square", metrics.r2_score(Y_test,Y_pred))
print("MAE",metrics.mean_absolute_error(Y_test,Y_pred))
print("MSE",metrics.mean_squared_error(Y_test,Y_pred))
print("RMSE",np.sqrt(metrics.mean_squared_error(Y_test,Y_pred)))

R-Square 0.9777690999899974
MAE 675.5374198721219
MSE 2120295.800197622
RMSE 1456.1235525179934


# when the model is deployed the ML algorithm helps to understand the patterns